In [115]:
import re
import json
from bs4 import BeautifulSoup as bs, NavigableString, Comment, Script
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Initialize and Read

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [107]:
# URL = "https://ant.design/components/form#register"
URL = "https://www.amazon.ca/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.ca%2F%3Fref_%3Dnav_ya_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=caflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&"
driver.get(URL)


In [108]:
# form = driver.find_element(By.ID, 'register')
form = driver.find_elements(By.TAG_NAME, 'form')[0]

In [109]:
doc = bs(form.get_attribute('outerHTML'), features='html.parser')

# Parse Tree

In [126]:
IGNORE_ATTRIBUTES = r'class|aria-\S*|router|\_ng\S*'
IGNORE_ATTRIBUTES = r''
IGNORE_TAGS = [
    'path', 'svg',
]
KEEP_TAGS = ['label']


def dfs_html_tree(root):
    global IGNORE_ATTRIBUTES, IGNORE_TAGS, KEEP_TAGS
    
    tag_name = root.name
    if tag_name in IGNORE_TAGS or type(root) == Comment or type(root) == Script:
        return None
    
    if type(root) == NavigableString:
        return root.text.strip()
    children = root.contents
    
    mapped_attrs = {
        x: root.attrs[x] for x in root.attrs if not re.match(IGNORE_ATTRIBUTES, x)
    }
    
    if len(children) == 0 or (len(children) == 1 and type(children[0]) == NavigableString):
        content = "" if len(children) == 0 else children[0].text
        return {
            "tag": tag_name,
            "attributes": root.attrs,
            "content": content
        }
    else:
        children = list(map(
            lambda child: dfs_html_tree(child),
            children
        ))
        children = list(filter(lambda x: x is not None and len(x) > 0, children))
        children = list(map(
            lambda child: child if type(child) == dict or len(child) > 1 else child[0],
            children
        ))
        
        if tag_name in KEEP_TAGS:
            return {
                "tag": tag_name,
                "attributes": root.attrs,
                "content": children if len(children) > 1 or type(children[0]) != str else children[0]
            }
        
        return children


parsed = dfs_html_tree(doc.contents[0])

# Remove Hidden Inputs

In [ ]:
def dfs_remove_hidden_inputs(root):
    if type(root) == dict and "type" in root["attributes"] and root["attributes"]["type"] == "hidden":
        return None
    elif type(root) == list:
        mapped = list(map(dfs_remove_hidden_inputs, root))
        mapped = list(filter(lambda x: x is not None, mapped))
        return mapped
    else:
        return root

In [124]:
with open('../parsed.json', 'w') as f:
    json.dump(parsed, f)